In [1]:
import os
import tensorflow as tf
from tensorflow.keras import datasets,layers,optimizers,Sequential,metrics
import numpy as np

In [2]:
#import os
#os.environ["CUDA_VISIBLE_DEVICES"] = ""

## Process ImageNet data

In [2]:
import tensorflow_datasets.public_api as dataset 
import tensorflow.keras.backend as backend

In [3]:
train_data,test_data=dataset.load("imagenette/320px-v2",split=["train","validation"],shuffle_files=True)

In [4]:
N=3000 #size
y_train=[]
start=True
c=0
for item in train_data:
    x = item['image']
    y = item['label']
    if start:
        x_train = tf.reshape(tf.image.resize(x,(224,224)), [1,224,224,3])
        start=False
    else:
        temp = tf.reshape(tf.image.resize(x,(224,224)), [1,224,224,3])
        x_train = tf.concat([x_train, temp], 0)
    y_train.append(backend.get_value(y))
    c+=1
    if c>N:
        break
x_train=x_train.numpy()
y_train=np.array(y_train)

In [5]:
N=500 #size
y_test=[]
start=True
c=0
for item in test_data:
    x = item['image']
    y = item['label']
    if start:
        x_test = tf.reshape(tf.image.resize(x,(224,224)), [1,224,224,3])
        start=False
    else:
        temp = tf.reshape(tf.image.resize(x,(224,224)), [1,224,224,3])
        x_test = tf.concat([x_test, temp], 0)
    y_test.append(backend.get_value(y))
    c+=1
    if c>N:
        break
x_test=x_test.numpy()
y_test=np.array(y_test)

## Test RestNet Model

In [8]:
from Resnet_func import *

In [9]:
model_resnet18 = build_ResNet('ResNet18', 10)
model_resnet18.build(input_shape=(None, 224, 224, 3))
model_resnet18.summary()

Model: "res_net"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              multiple                  9472      
_________________________________________________________________
sequential (Sequential)      (None, 56, 56, 64)        256       
_________________________________________________________________
sequential_1 (Sequential)    (None, 56, 56, 64)        148736    
_________________________________________________________________
sequential_2 (Sequential)    (None, 28, 28, 128)       526976    
_________________________________________________________________
sequential_3 (Sequential)    (None, 14, 14, 256)       2102528   
_________________________________________________________________
sequential_4 (Sequential)    (None, 7, 7, 512)         8399360   
_________________________________________________________________
global_average_pooling2d (Gl multiple                  0   

In [10]:
model_resnet18.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [11]:
model_resnet18.fit(x=x_train, y=y_train, batch_size=32, epochs=25, shuffle=True, verbose=1, validation_data=(x_test, y_test)) 

Epoch 1/25
94/94 [==============================] - 17s 124ms/step - loss: 2.8195 - accuracy: 0.2170 - val_loss: 40.5341 - val_accuracy: 0.0918
Epoch 2/25
94/94 [==============================] - 10s 107ms/step - loss: 1.8471 - accuracy: 0.3471 - val_loss: 2.3896 - val_accuracy: 0.2774
Epoch 3/25
94/94 [==============================] - 10s 108ms/step - loss: 1.6627 - accuracy: 0.4099 - val_loss: 42.2959 - val_accuracy: 0.1118
Epoch 4/25
94/94 [==============================] - 10s 109ms/step - loss: 1.4574 - accuracy: 0.5201 - val_loss: 12.9721 - val_accuracy: 0.1198
Epoch 5/25
94/94 [==============================] - 10s 111ms/step - loss: 1.3228 - accuracy: 0.5554 - val_loss: 7.4386 - val_accuracy: 0.1417
Epoch 6/25
94/94 [==============================] - 10s 111ms/step - loss: 1.1831 - accuracy: 0.5928 - val_loss: 3.4513 - val_accuracy: 0.3154
Epoch 7/25
94/94 [==============================] - 10s 111ms/step - loss: 1.1301 - accuracy: 0.6119 - val_loss: 2.5377 - val_accuracy: 0.3

## Test Resnet with ABC layers

#### Pre-train

In [12]:
from Resnet_func import *

In [13]:
model = build_ResNet('ResNet18', 10)
model.build(input_shape=(None, 224, 224, 3))
model.summary()

Model: "res_net_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_20 (Conv2D)           multiple                  9472      
_________________________________________________________________
sequential_5 (Sequential)    (None, 56, 56, 64)        256       
_________________________________________________________________
sequential_6 (Sequential)    (None, 56, 56, 64)        148736    
_________________________________________________________________
sequential_7 (Sequential)    (None, 28, 28, 128)       526976    
_________________________________________________________________
sequential_8 (Sequential)    (None, 14, 14, 256)       2102528   
_________________________________________________________________
sequential_9 (Sequential)    (None, 7, 7, 512)         8399360   
_________________________________________________________________
global_average_pooling2d_1 ( multiple                  0 

In [14]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [15]:
model.fit(x=x_train, y=y_train, batch_size=8, epochs=25, shuffle=True, verbose=1, validation_data=(x_test, y_test)) 

Epoch 1/25
376/376 [==============================] - 19s 45ms/step - loss: 2.7475 - accuracy: 0.1784 - val_loss: 3.2721 - val_accuracy: 0.1976
Epoch 2/25
376/376 [==============================] - 16s 42ms/step - loss: 2.0544 - accuracy: 0.2924 - val_loss: 4.0584 - val_accuracy: 0.2096
Epoch 3/25
376/376 [==============================] - 16s 43ms/step - loss: 1.8918 - accuracy: 0.3315 - val_loss: 3.1623 - val_accuracy: 0.2774
Epoch 4/25
376/376 [==============================] - 16s 42ms/step - loss: 1.7589 - accuracy: 0.3927 - val_loss: 2.3351 - val_accuracy: 0.3273
Epoch 5/25
376/376 [==============================] - 16s 42ms/step - loss: 1.6464 - accuracy: 0.4382 - val_loss: 2.0198 - val_accuracy: 0.3273
Epoch 6/25
376/376 [==============================] - 16s 42ms/step - loss: 1.5374 - accuracy: 0.4672 - val_loss: 3.9417 - val_accuracy: 0.1597
Epoch 7/25
376/376 [==============================] - 16s 42ms/step - loss: 1.4889 - accuracy: 0.4967 - val_loss: 1.6631 - val_accuracy:

#### Model Initialization

In [16]:
kernel_initialization=model.get_weights()

In [17]:
W=[]
bias=[]
i=0
W.append(tf.convert_to_tensor(kernel_initialization[0][0]))
bias.append(tf.convert_to_tensor(kernel_initialization[0][1]))
for item in kernel_initialization[1:]:
    W.append(item[0])
    W.append(item[2])
    bias.append(item[1])
    bias.append(item[3])    

In [18]:
from Resnet_self import *

In [19]:
B=[]
M=5
for i in W:
    Bi,_,_=get_binary_filter(i,M)
    B.append(Bi)

In [16]:
len(B)

17

In [17]:
tf.config.run_functions_eagerly(True)

In [20]:
N=3
tf.compat.v1.enable_eager_execution()
model_abc = build_ResNet('ResNet18', 10,M,N,B, bias, W)
#model_abc.build(input_shape=(None, 224, 224, 3))
#model_abc.summary()
model_abc.compile("adam",
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [21]:
model_abc.fit(x=x_train, y=y_train, batch_size=8, epochs=15, shuffle=True, verbose=1, validation_data=(x_test, y_test))

/home/ecbm4040/envTF24/lib/python3.6/site-packages/tensorflow/python/data/ops/dataset_ops.py:3504: UserWarning: Even though the tf.config.experimental_run_functions_eagerly option is set, this option does not apply to tf.data functions. tf.data functions are still traced and executed as graphs.
  "Even though the tf.config.experimental_run_functions_eagerly "


Epoch 1/15
376/376 [==============================] - ETA: 0s - loss: 2.4642 - accuracy: 0.1422

/home/ecbm4040/envTF24/lib/python3.6/site-packages/tensorflow/python/data/ops/dataset_ops.py:3504: UserWarning: Even though the tf.config.experimental_run_functions_eagerly option is set, this option does not apply to tf.data functions. tf.data functions are still traced and executed as graphs.
  "Even though the tf.config.experimental_run_functions_eagerly "


376/376 [==============================] - 132s 350ms/step - loss: 2.4640 - accuracy: 0.1422 - val_loss: 2.9162 - val_accuracy: 0.0798
Epoch 2/15
376/376 [==============================] - 131s 350ms/step - loss: 2.2547 - accuracy: 0.1649 - val_loss: 3.2986 - val_accuracy: 0.1198
Epoch 3/15
376/376 [==============================] - 132s 350ms/step - loss: 2.1912 - accuracy: 0.2088 - val_loss: 2.6103 - val_accuracy: 0.1138
Epoch 4/15
376/376 [==============================] - 131s 349ms/step - loss: 2.1222 - accuracy: 0.2319 - val_loss: 2.4280 - val_accuracy: 0.1637
Epoch 5/15
376/376 [==============================] - 131s 349ms/step - loss: 2.0856 - accuracy: 0.2546 - val_loss: 2.8730 - val_accuracy: 0.0978
Epoch 6/15
376/376 [==============================] - 131s 349ms/step - loss: 2.0385 - accuracy: 0.2784 - val_loss: 2.9240 - val_accuracy: 0.1537
Epoch 7/15
376/376 [==============================] - 132s 351ms/step - loss: 2.0190 - accuracy: 0.2854 - val_loss: 2.2377 - val_accura